Sources:
https://gist.github.com/eduardoftdo/e3d2b7ca4a06d8d86b144482d0aed5a1

http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install streamlit -q
!pip install efficient-apriori
#!pip install pyngrok
#%%writefile app.py

     |████████████████████████████████| 9.1 MB 17.9 MB/s 
     |████████████████████████████████| 78 kB 5.1 MB/s 
     |████████████████████████████████| 232 kB 47.5 MB/s 
     |████████████████████████████████| 4.3 MB 36.5 MB/s 
     |████████████████████████████████| 181 kB 67.9 MB/s 
     |████████████████████████████████| 164 kB 58.6 MB/s 
     |████████████████████████████████| 111 kB 53.0 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 132 kB 46.4 MB/s 
     |████████████████████████████████| 793 kB 43.8 MB/s 
     |████████████████████████████████| 132 kB 66.9 MB/s 
     |████████████████████████████████| 423 kB 41.4 MB/s 
     |████████████████████████████████| 381 kB 66.6 MB/s 
     |████████████████████████████████| 51 kB 5.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.6.6 requ

In [8]:
%%writefile streamlit_app.py 
import streamlit as st 
st.header('Association Rule Mining with Apriori Algorithm')
st.caption('Dataset: UCI Breast Cancer Data provided by the Oncology Institute')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os


from efficient_apriori import apriori

path = "/content/drive/MyDrive/breast-cancer.data"
#df = pd.read_csv(path)
data = pd.read_csv(path, header=None)
data.columns = ['Class', 'Age', 'Menopause', 'Tumor_Size', 'Inv_Nodes', 'Node_Caps', 'Malignancy_Degree', 'Breast', 'Breast_Quad', 'Irradiat']
data['Malignancy_Degree'] = data['Malignancy_Degree'].astype(str)

data['Class'] = pd.Categorical(data.Class)
data['Age'] = pd.Categorical(data.Age)
data['Menopause'] = pd.Categorical(data.Menopause)
data['Tumor_Size'] = pd.Categorical(data.Tumor_Size)
data['Inv_Nodes'] = pd.Categorical(data.Inv_Nodes)
data['Node_Caps'] = pd.Categorical(data.Node_Caps)
data['Malignancy_Degree'] = pd.Categorical(data.Malignancy_Degree)
data['Breast'] = pd.Categorical(data.Breast)
data['Breast_Quad'] = pd.Categorical(data.Breast_Quad)
data['Irradiat'] = pd.Categorical(data.Irradiat)

#data['Class', 'Age', 'Menopause', 'Tumor_Size', 'Inv_Nodes'] = pd.Categorical(data.Class, data.Age, data.Menopause, data.Tumor_Size, data.Inv_Nodes)
#data['Node_Caps', 'Malignancy_Degree', 'Breast', 'Breast_Quad', 'Irradiat'] = pd.Categorical(data.Node_Caps, data.Malignancy_Degree, data.Breast, data.Breast_Quad, data.Irradiat)

df = data

df["Irradiat"].replace({"yes": "Radiation", "no": "No_Radiation"}, inplace=True)
print(df['Irradiat'])


transactions = []
for i in range(0, 286):
 transactions.append([str(df.values[i,j]) for j in range(0,10)])

# Show dataframe
#transactions



from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
#df


df.rename(columns = {'0-2':'inv_nodes:0-2', '3-5':'inv_nodes:3-5', '6-8':'inv_nodes:6-8', '9-11':'inv_nodes:9-11','12-14':'inv_nodes:12-14', '15-17':'inv_nodes:15-17', '18-20':'inv_nodes:18-20', '21-23':'inv_nodes:21-23','24-26':'inv_nodes:24-26', 
                     '0-4':'tumor_size:0-4', '10-14':'tumor_size:10-14', '15-19':'tumor_size:15-19', '20-24':'tumor_size:20-24', '25-29':'tumor_size:25-29', '30-34':'tumor_size:30-34', '35-39':'tumor_size:35-39', '40-44':'tumor_size:40-44', '45-49':'tumor_size:45-49', '50-54':'tumor_size:50-54',
                     '20-29':'age:20-29', '30-39':'age:30-39', '40-49':'age:40-49', '50-59':'age:50-59', '60-69':'age:60-69', '70-79':'age:70-79',
                     'left':'breast:left', 'right':'breast:right',
                     'yes':'node_caps:yes', 'no':'node_caps:no',
                     '1':'deg_malig:1', '2': 'deg_malig:2', '3':'deg_malig:3',
                     'left_up':'breast_quad:left_up', 'left_low':'breast_quad:left_low', 'right_low':'breast_quad:right_low', 'central':'breast_quad:central',
                     'lt40':'menopause:lt40', 'ge40':'menopause:ge40', 'premeno':'menopause:premeno'}, inplace = True)



df.drop('?', axis=1, inplace=True)



from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#implementation of apriori general association rules without filter

st.subheader('1. Implementation of Apriori general association rules without filtering the consequents')


#Apriori min support
#min_support = 0.05
min_support = st.slider("Minimum Support Value", min_value=0.01,
                    max_value=0.9, value=0.15, key = 1)

#Max lenght of apriori itemsets
max_len = st.slider("Max lenght of apriori itemsets", min_value=1,
                    max_value=39, value=3, step=1, key = 2)

frequent_items = apriori(df, use_colnames=True, min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)

rules['antecedents'] = rules['antecedents'].astype('string')
rules['consequents'] = rules['consequents'].astype('string')

#select values to display
rules = rules[['antecedents', 'consequents', 'support', 'lift', 'confidence', 'conviction']]
rules



#Filtering only consequents with 'Recurrence Events' or 'No Recurrent Events'


st.subheader('2. Filtering only consequents with Recurrence Events or Non-Recurrence Events')

#slider to selecet a min support value
min_support = st.slider("Minimum Support Value", min_value=0.01,
                    max_value=0.9, value=0.10, key = 3)

#Max lenght of apriori itemsets
max_len = st.slider("Max. lenght of apriori itemset", min_value=1,
                    max_value=39, value=3, step=1, key =4)

frequent_items = apriori(df, use_colnames=True, min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)


#target = '{\'recurrence-events\'}'
target = st.selectbox(
     'Select a target:',
     ('recurrence-events', 'no-recurrence-events'))
target = "{'" + target + "'}"
     
#target = target.astype(str)
results_target = rules[rules['consequents'].astype(str).str.contains(target, na=False)].sort_values(by='confidence', ascending=False)
results_target['antecedents'] = results_target['antecedents'].astype('string')
results_target['consequents'] = results_target['consequents'].astype('string')
#results_target.head(10)

results_target = results_target[['antecedents', 'consequents', 'support', 'lift', 'confidence', 'conviction']]


results_target


Writing streamlit_app.py


In [ ]:
!streamlit run streamlit_app.py & npx localtunnel --port 8501 

2022-07-10 22:39:42.818 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
npx: installed 22 in 6.114s
your url is: https://silly-pans-find-35-188-229-76.loca.lt

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.188.229.76:8501

0      No_Radiation
1      No_Radiation
2      No_Radiation
3      No_Radiation
4      No_Radiation
           ...     
281    No_Radiation
282       Radiation
283    No_Radiation
284    No_Radiation
285    No_Radiation
Name: Irradiat, Length: 286, dtype: object
